In [1]:
# 


In [1368]:
import geopandas as gpd
import json
import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors
from datetime import timedelta
import matplotlib.pyplot as plt
import networkx as nx
from shapely.geometry import Point
from shapely.geometry import LineString
from shapely.geometry import MultiLineString
import numpy as np
import numpy as npm
import osmnx as ox
import math
import pandas as pd
from shapely.ops import snap
from tqdm import tqdm
import re

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [1369]:
avg_speed_df_cache = {}
avg_speed_attr_dict_cache = {}

In [1370]:
CRS_LATLON = 'EPSG:4326'
DATA_DIR = '../../data'
EXPORTS_DIR = f'{DATA_DIR}/exports'

In [1371]:
def peek(df):
    print(len(df))
    display(df.iloc[0:3])

In [1372]:
trip_stop_sequence_dict = {}
with open(f'{EXPORTS_DIR}/json/manhattan/trip_stop_sequence_dict.json', 'r') as fp:
    trip_stop_sequence_dict = json.load(fp)

In [1373]:
trip_manifest = {}
with open(f'{EXPORTS_DIR}/json/manhattan/trip_manifest.json', 'r') as fp:
    trip_manifest = json.load(fp)

In [1374]:
timetable_df = pd.read_csv(f'{EXPORTS_DIR}/csv/manhattan/timetable.csv')
peek(timetable_df)

505068


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,trip_headsign,shape_id
0,OF_C1-Weekday-033500_M1_101,05:35:00,05:35:00,400001,1,0,0,HARLEM 147 ST via MADISON AV,M010006
1,OF_C1-Weekday-033500_M1_101,05:35:36,05:35:36,400002,2,0,0,HARLEM 147 ST via MADISON AV,M010006
2,OF_C1-Weekday-033500_M1_101,05:36:14,05:36:14,400003,3,0,0,HARLEM 147 ST via MADISON AV,M010006


In [1375]:
stops_nodes_df = pd.read_csv(f'{EXPORTS_DIR}/csv/manhattan/stops_nodes.csv')
stops_nodes_df = stops_nodes_df.set_index('stop_id')
peek(stops_nodes_df)

132


,stop_name,stop_lat,stop_lon,M1_0,M1_1
stop_id,,,,,
400001,4 AV/E 10 ST,40.731342,-73.990292,POINT (-73.99240 40.73060),NaN
400002,4 AV/E 12 ST,40.732608,-73.989958,POINT (-73.99240 40.73060),NaN
400003,4 AV/E 13 ST,40.733936,-73.989720,POINT (-73.99240 40.73060),NaN


In [1376]:
route_ids = list(stops_nodes_df.columns[3:])

In [1377]:
from shapely import wkt

route_gdfs = []

for route_id in route_ids:
    route_df = stops_nodes_df[~stops_nodes_df[route_id].isna()]
    route_df[route_id] = route_df[route_id].apply(wkt.loads)
    route_df = route_df[[route_id]]
    route_gdf = gpd.GeoDataFrame(route_df, geometry=route_df[route_id], crs=CRS_LATLON)
    route_gdfs.append(route_gdf)


In [1378]:
def get_distance(row):
    if row['last_geometry'] is None:
        return None
    # Approx degrees to meters
    return row['geometry'].distance(row['last_geometry']) * (0.11 / 0.000001)

In [1379]:
def get_point_df(trip_key, route_id):
    sequence_key = trip_key[-1]
    sequence = trip_stop_sequence_dict[trip_key]
    
    sequence_df = pd.DataFrame({'stop_id': sequence})
    sequence_df = sequence_df.set_index('stop_id')

    route_index = route_ids.index(route_id)
    route_gdf = route_gdfs[route_index]

    sequence_df = sequence_df.merge(route_gdf, left_index=True, right_index=True, how='left')
    
    sequence_gdf = gpd.GeoDataFrame(sequence_df, crs=CRS_LATLON)
    sequence_gdf = sequence_gdf[['geometry']]
    
    return sequence_gdf

In [1380]:
def get_distance_df(trip_key):
    sequence_key = trip_key[-1]
    sequence = trip_stop_sequence_dict[trip_key]
    
    sequence_df = pd.DataFrame({'stop_id': sequence})
    sequence_df = sequence_df.set_index('stop_id')

    route_index = route_ids.index(route_id)
    route_gdf = route_gdfs[route_index]

    sequence_df = sequence_df.merge(route_gdf, left_index=True, right_index=True, how='left')
    sequence_df = sequence_df.drop(columns=[route_id])
    
    sequence_gdf = gpd.GeoDataFrame(sequence_df, crs=CRS_LATLON)
    sequence_gdf['last_geometry'] = sequence_gdf['geometry'].shift()
    sequence_gdf['distance'] = sequence_gdf.apply(lambda x: get_distance(x), axis=1)
    sequence_gdf
    
    return sequence_gdf

In [1381]:
trip_manifest = {}
with open(f'{EXPORTS_DIR}/json/manhattan/trip_manifest.json', 'r') as fp:
    trip_manifest = json.load(fp)

In [1382]:
def get_duration(row):
    try:
        start_time = row['last_time']
        end_time = row['time']

        if int(start_time[0:2]) > 23 or int(end_time[0:2]) > 23:
            start_time = f'{int(start_time[0:2]) - 12}{start_time[2:]}'
            end_time = f'{int(end_time[0:2]) - 12}{end_time[2:]}'
    
        return pd.Timedelta(pd.to_datetime(end_time, format='%H:%M:%S') - pd.to_datetime(start_time, format='%H:%M:%S')).seconds
    except:
        return None

In [1383]:
def get_times_df(trip_id):
    timetable_mi_df = timetable_df[timetable_df['trip_id'].str.contains(trip_id)]
    timetable_mi_df = timetable_mi_df[timetable_mi_df['trip_id'].str.contains(f'_{route_id}_')]
    timetable_mi_df = timetable_mi_df.set_index(['trip_id', 'stop_id'])
    timetable_mi_df = timetable_mi_df.rename(columns={'arrival_time': 'time'})
    
    times_df = timetable_mi_df.loc[trip_id]
    times_df['last_time'] = times_df['time'].shift()
    times_df['duration'] = times_df.apply(lambda x: get_duration(x), axis=1)
    times_df = times_df
    return times_df

In [1384]:
import ast

route_mgs = []

for route_id in route_ids:
    route_mg = nx.read_graphml(f'{EXPORTS_DIR}/hybrid/manhattan/routes/{route_id}.graphml')
    
    g = nx.DiGraph()
    edges = [(ast.literal_eval(e[0]), ast.literal_eval(e[1])) for e in route_mg.edges]
    g.add_edges_from(edges)
    g.graph['crs'] = CRS_LATLON
    route_mgs.append(g)

In [1385]:
def graph_to_node_gdf(G):
    """Converts the given graph to a GeoDataFrame of its nodes."""
    nodes, data = zip(*G.nodes(data=True))
    geom = [Point(node[0], node[1]) for node in nodes]
    nodes_gdf = gpd.GeoDataFrame(data,
                                 index=list(nodes),
                                 crs=CRS_LATLON,
                                 geometry=geom)
    return nodes_gdf

In [1386]:
def graph_to_edge_gdf(G):
    """Converts the given graph to a GeoDataFrame of its edges."""
    u, v, data = zip(*G.edges(data=True))
    edges = list(zip(u, v))
    attrs = nx.get_edge_attributes(G, 'speed')
    geom = [
        LineString((Point(edge[0][0],
                          edge[0][1]), Point(edge[1][0], edge[1][1])))
        for edge in edges
    ]
    edge_index = [(edge[0] + edge[1]) for edge in edges]
    edges_gdf = gpd.GeoDataFrame(data,
                                 index=edge_index,
                                 crs=CRS_LATLON,
                                 geometry=list(geom))
    edges_gdf['length'] = edges_gdf.geometry.length
    edges_gdf = edges_gdf.drop(columns={
        'route_shor',
        'route_long',
        'color',
        'ShpName',
        'Wkb',
        'Wkt',
        'Json',
    }, errors='ignore')
    return edges_gdf

In [1387]:
def get_node(mg, point):
    threshold = 0.00001
    for node in route_mg.nodes():
        if abs(node[0] - point[0]) < threshold and abs(node[1] - point[1]) < threshold:
            return node
    return None

In [1388]:
def get_trip_ids(trip_key):
    route_id = trip_key.split(',')[0]
    trip_id = trip_key.split(',')[1]
    trip_keys = list(trip_manifest[trip_key])
    trip_keys = [f for f in trip_keys if re.match(f'{trip_id}.*', f)]
    trip_keys = [f for f in trip_keys if re.match(f'.*_{route_id}_', f)]
    return trip_keys

In [1389]:
hours = list(range(3))

In [1390]:
def get_hour(x):
    return int(math.floor((int(x[0:2]) % 24) / 8))

In [1391]:
def get_speed_df(trip_id, point_df, times_df):
    speed_df = times_df.merge(point_df, left_index=True, right_index=True, how='left')
    speed_df = speed_df[['time', 'stop_sequence', 'geometry']]
    speed_df = speed_df[speed_df['geometry'].notnull()]
    speed_df = speed_df.sort_values(by=['stop_sequence'])
    speed_df['last_time'] = speed_df['time'].shift()
    speed_df['last_geometry'] = speed_df['geometry'].shift()
    
    if len(speed_df) == 0:
        return None
    
    speed_df['distance'] = speed_df.apply(lambda x: get_distance(x), axis=1)
    speed_df['duration'] = speed_df.apply(lambda x: get_duration(x), axis=1)
    speed_df['speed'] = speed_df['distance'] / speed_df['duration']
    speed_df = speed_df.drop(columns=['stop_sequence', 'last_time', 'distance', 'duration'])
    speed_df = speed_df.dropna()
    speed_df['hour'] = speed_df['time'].apply(lambda x: get_hour(x))
    speed_df = speed_df.rename(columns={'last_geometry': 'start', 'geometry': 'end'})
    return speed_df

In [1406]:
def get_avg_speed_dfs(trip_key, route_id):
    point_df = get_point_df(trip_key, route_id)
    speed_dfs = []
    errors = 0
    
    trip_ids = get_trip_ids(trip_key)

    for trip_id in trip_ids:
        times_df = get_times_df(trip_id)
        speed_df = get_speed_df(trip_id, point_df, times_df)
        if speed_df is None:
            continue
        
        speed_df['start'] = speed_df['start'].apply(lambda x: x.wkt)
        speed_df['end'] = speed_df['end'].apply(lambda x: x.wkt)
        speed_df = speed_df.reset_index()
        speed_df = speed_df.drop(columns=['stop_id'])
        speed_df = speed_df.set_index(['start', 'end'])
        speed_dfs.append(speed_df)
        
    avg_speed_dfs = []
    
    if len(speed_dfs) == 0:
        raise Exception(f'{trip_key} produced no speed_dfs')
    
    for hour in hours:
        hour_dfs = [df[df['hour'] == hour].drop(columns=['hour', 'time']) for df in speed_dfs]
        
        all_speed_dfs = hour_dfs[0].join(hour_dfs[1:], how='outer')
        
        avg_speed_df = pd.DataFrame(all_speed_dfs.mean(axis=1))
        avg_speed_dfs.append(avg_speed_df)
        
    return avg_speed_dfs

In [1407]:
def get_avg_speed_attr_dict(avg_speed_dfs, route_mg):
    attr_dict = {}
    for hour, avg_speed_df in enumerate(avg_speed_dfs):
        for row in avg_speed_df.iterrows():
            index = row[0]
            start, end = (wkt.loads(f) for f in index)
            avg_speed = row[1][0]

            start_node = get_node(route_mg, (start.x, start.y))
            end_node = get_node(route_mg, (end.x, end.y))

            try:
                path = nx.shortest_path(route_mg, end_node, start_node)
            except:
                continue

            for i in range(len(path) - 1):
                edge = (path[i], path[i+1])
                if edge not in attr_dict:
                    attr_dict[edge] = {}
                key = f'speeds_{hour}'
                if key not in attr_dict[edge]:
                    attr_dict[edge][key] = []
                attr_dict[edge][key].append(avg_speed)
                
        print(hour, len(avg_speed_df))
        
    return attr_dict

In [1408]:
#list(trip_stop_sequence_dict.keys())
#del avg_speed_df_cache['M3,MV_C1-Weekday,1']

In [1420]:
trip_keys = [
    'M1,OF_C1-Weekday,0',
    'M1,OF_C1-Weekday,1',
    #'M2,MV_C1-Weekday,0',
    #'M2,MV_C1-Weekday,1',
    #'M3,MV_C1-Weekday,0',
    #'M3,MV_C1-Weekday,1',
    #'M4,MV_C1-Weekday,0',
    #'M4,MV_C1-Weekday,1',
    #'M57,MQ_C1-Weekday,0',
    #'M57,MQ_C1-Weekday,1',
    #'M1,OF_C1-Weekday,0',
    #'M1,OF_C1-Weekday,1',
    #'M2,MV_C1-Weekday,0',
    #'M2,MV_C1-Weekday,1',
    #'M3,MV_C1-Weekday,0',
    #'M3,MV_C1-Weekday,1',
    #'M4,MV_C1-Weekday,0',
    #'M4,MV_C1-Weekday,1',
]

In [1421]:
#trip_keys = list(trip_stop_sequence_dict.keys())[0:10]

avg_speed_dfs = []
avg_speed_attr_dicts = []
errors = 0
error_trips = []

uncached_trip_keys = []

for trip_key in trip_keys:
    if trip_key in avg_speed_df_cache:
        avg_speed_dfs.append(avg_speed_df_cache[trip_key])
        avg_speed_attr_dicts.append(avg_speed_attr_dict_cache[trip_key])
    else:
        uncached_trip_keys.append(trip_key)

#for trip_key in tqdm(uncached_trip_keys):
for trip_key in uncached_trip_keys:
    #try:
    route_id = trip_key.split(',')[0]
    trip_id = trip_key.split(',')[1]
    sequence_id = trip_key.split(',')[2]
    full_key = f'{route_id}_{sequence_id}'
    route_mg = route_mgs[route_ids.index(full_key)]

    avg_speed_dfs = get_avg_speed_dfs(trip_key, full_key)
    avg_speed_attr_dict = get_avg_speed_attr_dict(avg_speed_dfs, route_mg)

    avg_speed_dfs.append(avg_speed_df)
    avg_speed_attr_dicts.append(avg_speed_attr_dict)

    avg_speed_df_cache[trip_key] = avg_speed_df
    avg_speed_attr_dict_cache[trip_key] = avg_speed_attr_dict
    #except:
    #    errors += 1
    #    error_trips.append(trip_key)
print(f'{len(trip_keys) - errors}/{len(trip_keys)}')
display(error_trips)

KeyboardInterrupt: 

In [ ]:
avg_speed_attr_dict = {}
for attr_dict in avg_speed_attr_dicts:
    for key, value in attr_dict.items():
        if key not in avg_speed_attr_dict:
            avg_speed_attr_dict[key] = {}
        for hour_key in value.keys():
            if hour_key not in avg_speed_attr_dict[key]:
                avg_speed_attr_dict[key][hour_key] = []
                avg_speed_attr_dict[key][hour_key] += value[hour_key]
        
for key, value in avg_speed_attr_dict.items():
    for hour_key in value.keys():
        avg_speed_attr_dict[key][hour_key] = np.mean(value[hour_key])

In [ ]:
all_routes_mg = route_mgs[0]

for i, route_mg in enumerate(route_mgs[1:]):
    all_routes_mg = nx.compose(all_routes_mg, route_mg)

In [ ]:
nx.set_edge_attributes(all_routes_mg, avg_speed_attr_dict)

In [ ]:
all_nodes_gdf = graph_to_node_gdf(all_routes_mg)
all_edges_gdf = graph_to_edge_gdf(all_routes_mg)

In [ ]:
gdf = gpd.GeoDataFrame({}, geometry=[Point(-73.93606, 40.82071), Point(-73.93536, 40.82177)], crs=CRS_LATLON)

In [ ]:
fig, ax = plt.subplots(figsize=(50, 50))
all_edges_gdf.plot(ax=ax, color='#333', zorder=1)
all_edges_gdf.plot(ax=ax, column='speeds_2', cmap='RdBu', zorder=2, linewidth=3, legend=True)
gdf.plot(ax=ax, markersize=50, color='#000', zorder=5)
ax.set_facecolor('#777')

In [ ]:
len(list(trip_stop_sequence_dict.keys()))

In [ ]:
import pickle
f = open(f'{EXPORTS_DIR}/pkl/avg_speed_attr_dict.pkl', 'wb')
pickle.dump(avg_speed_attr_dict, f)
f.close()